In [1]:
import pandas as pd
import numpy as np
import os
import glob
import sys
import shutil

In [3]:
#path to folder with all subjects data

path = '/media/data/HCPAging/data/VISMOTORTaskRecommended/fmriresults01/'
#outputs
os.makedirs('/media/data/HCPAging/data/3T_tfMRI_VISMOTOR')
path_out = '/media/data/HCPAging/data/3T_tfMRI_VISMOTOR/'

In [4]:
#read subjects
subjects = sorted([name.split('/')[-1] for name in glob.glob(path+'*_V1_MR')])
print(len(subjects))
#print(subjects)

721


### Movements

In [5]:
#Assembling movements into one
vec = np.array([], dtype=float)
nms=[]
for subject in subjects:
    file =  glob.glob(path + subject +'/*/*/*/Movement_RelativeRMS_mean*')[0]
    if os.path.isfile(file) == True:
        nms += [subject]
        vec = np.append(vec, np.loadtxt(file))
    else:
        print(subject, 'issue')
df_mov = pd.Series(vec, index=nms)  
df_mov.to_csv(path_out+'movement_table_VISMOTOR_without_nan.csv', header=None)

### CIFTI files

In [6]:
#subjects without nan
subjects = sorted(df_mov.dropna(axis=0).index)

In [7]:
len(subjects)

721

In [10]:
#subdirectory for processing
os.mkdir(path_out+'data')

##### create fakeNIFTI file 

In [ ]:
#subject=subjects[0]
for subject in subjects:
    #check if subject has cifti file and events files
    if os.path.isfile(glob.glob(path+subject+'/*/*/*/*Atlas_MSMAll_hp0_clean.dtseries.nii')[0]) == True and os.path.isdir(glob.glob(path+subject+'/*/*/*/EVs')[0])==True:
        os.mkdir(path_out+'data/'+subject) #subject folder

        #create fakeNIFTI
        cmd1= 'wb_command -cifti-convert -to-nifti ' + glob.glob(path+subject+'/*/*/*/*Atlas_MSMAll_hp0_clean.dtseries.nii')[0] +' '+ path_out+'data/'+subject+'/tfMRI_MSMAll_clean_fakeNIFTI.nii.gz'
        os.system(cmd1)
        #create a template for reverse trensformation
        cmd2='wb_command -cifti-reduce ' + glob.glob(path+subject+'/*/*/*/*Atlas_MSMAll_hp0_clean.dtseries.nii')[0]  + ' MEAN ' + path_out+'data/'+subject+'/template_cifti_aver.dscalar.nii' + ' -direction  ROW'
        os.system(cmd2)
        #change TR
        cmd3 = 'fslmerge -tr '+ path_out+'data/'+subject+'/tfMRI_MSMAll_clean_fakeNIFTI.nii.gz' +' '+ path_out+'data/'+subject+'/tfMRI_MSMAll_clean_fakeNIFTI.nii.gz' + ' 0.8'
        os.system(cmd3)
        #copy event files
        cmd4 = 'cp -R ' + glob.glob(path+subject+'/*/*/*/EVs')[0] +' '+ path_out+'data/'+subject+'/EVs'
        os.system(cmd4)
    else:
        print(subjects, ' has missing things')